In [1]:
import os
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tiff
from matplotlib.pyplot import figure

basePath = "/home/cov/Desktop/PML/project1_Mura/AUO_Data/2nd/0826_2nd/"
csvPath = basePath + "table2.csv"

In [10]:
lst = pd.read_csv(csvPath)
# RealJND = []

for ii in range(1,21,1):
    imgPath = basePath + lst.Deftype[ii] +'/'+ lst.Chip_ID[ii] +'/'
    # RealJND.append(lst.RealJND[ii])
    if lst.Deftype[ii] == 'WSL128':
        for file_name in os.listdir(imgPath):
            if ("L128" in file_name) and (".tif" in file_name):
                fname = imgPath + file_name
                img1 = tiff.imread(fname)
#                 figure(figsize=(16, 12), dpi=80)
#                 plt.imshow(img1, cmap='gray')
#                 plt.show()
#                 print(fname)
            if ".bmp" in file_name:
                fname = imgPath + file_name
                img2 = cv2.imread(fname)
#                 figure(figsize=(16, 12), dpi=80)
#                 plt.imshow(img2, cmap='gray')
#                 plt.show()
#                 print(fname)
    print("===============")
    indices = np.where(np.all(img2 == [0,0,255], axis=-1))
    coords = zip(indices[0], indices[1])
    a = list(coords)[0]
    coords = zip(indices[0], indices[1])
    b = list(coords)[-1]
    
    imgFloat = img1.astype('float')
    coeff = cv2.dct(imgFloat)
    coeff[:,3:]=0
    coeff[3:][:]=0
    reconsImg = cv2.idct(coeff)
    diff = reconsImg-img1
    
    img1 = img1[2*a[0]:2*b[0],2*a[1]:2*b[1]]
    diff = diff[2*a[0]:2*b[0],2*a[1]:2*b[1]]
    
    diff_n = cv2.normalize(diff, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC1)
    thresh1 = cv2.adaptiveThreshold(diff_n, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 45, 22)# obtain white Mura
    thresh2 = cv2.adaptiveThreshold(diff_n, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 45,-22)# obtain black Mura
    BothT = thresh1+thresh2
    BothT2 = cv2.medianBlur(BothT, 9)
    kernel = np.ones((2,2),np.uint8)
    opening = cv2.morphologyEx(BothT2, cv2.MORPH_OPEN, kernel)
    
    color = (255,200,200)#(200,200,200)
    lst_dfcts = []
    lst_bdbox = []
    bdRect = []

    _, opening2 = cv2.threshold(opening, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(opening2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for i in range(len(contours)):
        cimg,cimg2 = np.zeros_like(opening), np.zeros_like(opening)
        cv2.drawContours(cimg, contours, i, color, -1, cv2.LINE_8)
        pts = np.where(cimg == 255)
        lst_dfcts.append(img1[pts[0], pts[1]]) # record the intensity at the defect pixels
        bdRect.append(cv2.boundingRect(contours[i]))
        cv2.rectangle(cimg2, (int(bdRect[i][0]), int(bdRect[i][1])),\
                      (int(bdRect[i][0]+bdRect[i][2]), int(bdRect[i][1]+bdRect[i][3])), 255, -1)
        pts2 = np.where(cimg2 == 255)
        lst_bdbox.append(img1[pts2[0], pts2[1]])

        M = cv2.moments(contours[i])
        if M["m00"]!=0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            print(cX,cY)
            
            
    total_pixel_count = img1.shape[0]*img1.shape[1]
    defect_pixel_count = 0
    for i in range(len(lst_dfcts)):
        defect_pixel_count += len(lst_dfcts[i])

    defect_intens = 0
    for i in range(len(lst_dfcts)):
        defect_intens += sum(lst_dfcts[i])
    total_intens = cv2.sumElems(img1)[0] - defect_intens

    I_Back = total_intens/(total_pixel_count-defect_pixel_count)

    Contrast = (img1[cX,cY]-I_Back)/(img1[cX,cY]+I_Back)
    print(str(Contrast)+", "+str(lst.RealJND[ii]))
    
#     elif lst.Deftype[i] == 'WSL48':
#         for file_name in os.listdir(imgPath):
#             if ("L48" in file_name) and (".tif" in file_name):
#                 fname = imgPath + file_name
#                 img = tiff.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
#             if ".bmp" in file_name:
#                 fname = imgPath + file_name
#                 img = cv2.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
                
#     elif lst.Deftype[i] == 'DWL48':
#         for file_name in os.listdir(imgPath):
#             if ("L48" in file_name) and (".tif" in file_name):
#                 fname = imgPath + file_name
#                 img = tiff.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
#             if ".bmp" in file_name:
#                 fname = imgPath + file_name
#                 img = cv2.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
                
#     elif lst.Deftype[i] == 'BSL48':
#         for file_name in os.listdir(imgPath):
#             if ("L48" in file_name) and (".tif" in file_name):
#                 fname = imgPath + file_name
#                 img = tiff.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
#             if ".bmp" in file_name:
#                 fname = imgPath + file_name
#                 img = cv2.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
                
#     elif lst.Deftype[i] == 'BSL128':
#         for file_name in os.listdir(imgPath):
#             if ("L128" in file_name) and (".tif" in file_name):
#                 fname = imgPath + file_name
#                 img = tiff.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
#             if ".bmp" in file_name:
#                 fname = imgPath + file_name
#                 img = cv2.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
                
#     elif lst.Deftype[i] == 'IRR':
#         for file_name in os.listdir(imgPath):
#             if ("L128" in file_name) and (".tif" in file_name):
#                 fname = imgPath + file_name
#                 img = tiff.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()
#             if ".bmp" in file_name:
#                 fname = imgPath + file_name
#                 img = cv2.imread(fname)
#                 plt.imshow(img, cmap='gray')
#                 plt.show()


56 57
0.03876723482889213, 3.1
202 73
54 58
0.021592694638001064, 2.6
56 58
0.049473824799867214, 2.9
113 161
0.13260749613258, 2.8
130 135
56 55
0.056111741491609336, 2.8
57 55
0.0405719480468757, 2.8
104 200
96 178
135 170
226 147
182 107
97 95
83 72
58 58
0.042219940494330976, 2.8
56 55
0.024076633692882803, 2.7
58 59
0.01643896235785174, 2.6
57 56
0.046991265857534346, 2.8
111 144
0.17892177873881251, 2.8
6 163
68 99
104 62
55 54
0.008834178131788531, 2.9
56 57
0.049157397265205176, 3.1
57 56
0.03309069831630295, 2.6
57 57
0.03477361694192059, 2.4
82 161
54 55
0.009672210875615674, 2.6
60 59
0.04282649413897099, 2.5
56 55
0.07929244598010868, 2.9
95 209
49 168
92 138
181 129
109 130
78 121
146 119
77 85
44 84
122 73
116 73
112 73
111 68
60 73
54 54
0.023049048079683946, 2.6
116 127
0.23689323020787917, 2.8


In [8]:
for ii in range(21):
    print(lst.RealJND[ii])

3.4
3.1
2.6
2.9
2.8
2.8
2.8
2.8
2.7
2.6
2.8
2.8
2.9
3.1
2.6
2.4
2.6
2.5
2.9
2.6
2.8
